# Phase DV Chip - Detecting real life Violence using CV

In [1]:
import os
import cv2
import numpy as np
from imutils import paths

In [2]:
from sklearn.preprocessing import LabelBinarizer

# Importing datapath 

In [3]:
datapath="video dataset"
outputmodel="output"
outputlabelbinarizer="output\videoclassificationbinarizer"

In [4]:
import cv2
import os
import numpy as np
import pickle
import matplotlib
matplotlib.use("Agg")

from keras.layers import Input
from keras.models import Model
from keras.layers.core import Dropout,Flatten,Dense

import matplotlib.pyplot as plt

In [5]:
mean = np.array([123.68, 116.779, 103.939][::1], dtype="float32")

In [6]:
cv2.__version__

'4.5.2'

# Predict video for Violence/Non-Violence

In [7]:

import argparse
import pickle
import cv2
import os
import time 
from keras.models import load_model
from collections import deque

def print_results(video, limit=None):
        #fig=plt.figure(figsize=(16, 30))
        if not os.path.exists('output'):
            os.mkdir('output')

        print("Loading model ...")
        model = load_model('model (3).h5')
        Q = deque(maxlen=128)
        vs = cv2.VideoCapture(video)
        writer = None
        (W, H) = (None, None)
        count = 0     
        while True:
            # read the next frame from the file
            (grabbed, frame) = vs.read()

            # if the frame was not grabbed, then we have reached the end
            # of the stream
            if not grabbed:
                break
            
            # if the frame dimensions are empty, grab them
            if W is None or H is None:
                (H, W) = frame.shape[:2]

            # clone the output frame, then convert it from BGR to RGB
            # ordering, resize the frame to a fixed 128x128, and then
            # perform mean subtraction

            
            output = frame.copy()
           
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, (128, 128)).astype("float32")
            frame = frame.reshape(128, 128, 3) / 255 #feature scaling 
#             frame -= mean

            # make predictions on the frame and then update the predictions
            # queue
            preds = model.predict(np.expand_dims(frame, axis=0))[0]
            print("preds",preds)
            Q.append(preds)

            # perform prediction averaging over the current history of
            # previous predictions
#             results = np.array(Q).mean(axis=0)
#             i = np.argmax(results)
            i = (preds > 0.50)[0]
#             print(i)
            label = i


            # draw the activity on the output frame
            # prob = model.predict_proba(np.expand_dims(frame, axis=0))[0] # to show probability of frame
#             prob = results[i]*100
#             print(prob)

            text_color = (0, 255, 0) # default : green

            if label: # Violence prob
                text_color = (0, 0, 255) # red

            else:
                text_color = (0, 255, 0)

            #text = "State : {:8} ({:3.2f}%)".format(label,prob)
            text = "Violence: {}".format(label)
            FONT = cv2.FONT_HERSHEY_SIMPLEX 

            cv2.putText(output, text, (35, 50), FONT,1.25, text_color, 3) 

            # plot graph over background image
#             output = cv2.rectangle(output, (35, 80), (35+int(prob)*5,80+20), text_color,-1)

            # check if the video writer is None
            if writer is None:
                # initialize our video writer
                fourcc = cv2.VideoWriter_fourcc(*"MJPG")
                writer = cv2.VideoWriter("output/NV_OUTPUT.avi", fourcc, 30,(W, H), True)

            # write the output frame to disk
            writer.write(output)

            # show the output image
            cv2.imshow("Output", output)
            key = cv2.waitKey(1) & 0xFF

            # if the `q` key was pressed, break from the loop
            if key == ord("q"):
                break
        # release the file pointersq
        print("[INFO] cleaning up...")
        writer.release()
        vs.release()


In [8]:
Violence="video dataset/Violence/V_1.mp4"

In [11]:
Nonviolence="video dataset/NonViolence/NV_23.mp4"

# Output

In [12]:
 print_results(Nonviolence) 

Loading model ...
preds [0.01777709]
preds [0.0036068]
preds [0.00569719]
preds [0.00584787]
preds [0.01834029]
preds [0.01184785]
preds [0.01605573]
preds [0.02007926]
preds [0.0204767]
preds [0.0195708]
preds [0.01302612]
preds [0.01073775]
preds [0.00386789]
preds [0.00616801]
preds [0.00287417]
preds [0.00298986]
preds [0.00101042]
preds [0.00105086]
preds [0.00402552]
preds [0.0036093]
preds [0.00499463]
preds [0.00483802]
preds [0.00289133]
preds [0.00077283]
preds [0.00160471]
preds [0.00170532]
preds [0.00459918]
preds [0.00502861]
preds [0.01482373]
preds [0.00436467]
preds [0.00414601]
preds [0.0130707]
preds [0.0055179]
preds [0.00538343]
preds [0.00307658]
preds [0.00552428]
preds [0.00558707]
preds [0.00644127]
preds [0.00424647]
preds [0.00417534]
preds [0.00753888]
preds [0.0027076]
preds [0.00173974]
preds [0.00375986]
preds [0.00141084]
preds [0.00137705]
preds [0.00066218]
preds [0.00088677]
preds [0.00307772]
preds [0.00319871]
preds [0.00209162]
preds [0.00209045]
p